# Импорт библиотек

In [110]:
# !pip install catboost
import pandas as pd
import json
from geopy.distance import geodesic
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from tqdm import tqdm
import re
from sklearn.linear_model import Lasso
import xgboost as xgb
import lightgbm as lgb

# Загрузка и обработка тренировочных данных

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Шаг 1: Загрузка данных из JSON файла
file_path = '/content/drive/My Drive/train_data.json'  # укажите путь к вашему JSON файлу

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Шаг 2: Преобразование данных в DataFrame
df = pd.json_normalize(data, 'points', ['hash', 'targetAudience', 'value'])

# Преобразование целевой аудитории в отдельные столбцы
df = df.join(pd.json_normalize(df.pop('targetAudience')))
df

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id
0,55.573691,37.631423,273,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN
1,55.584765,37.712454,232,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN
2,55.808425457052,37.388807961811,188,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN
3,55.674378,37.422364,333,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN
4,55.608396,37.766383,54,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN
...,...,...,...,...,...,...,...,...,...,...,...
136834,55.907413,37.53523,259,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN
136835,55.871245,37.410527,231,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN
136836,55.585678,37.715744,56,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN
136837,55.74793358781934,37.53652274608613,224,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN


In [5]:
data_train = df[['hash','name', 'gender', 'ageFrom', 'ageTo','income', 'lat', 'lon', 'azimuth', 'value']]
data_train

,hash,name,gender,ageFrom,ageTo,income,lat,lon,azimuth,value
0,41567f28db47bee7,All 25-45 BC,all,25,45,bc,55.573691,37.631423,273,23.51
1,41567f28db47bee7,All 25-45 BC,all,25,45,bc,55.584765,37.712454,232,23.51
2,41567f28db47bee7,All 25-45 BC,all,25,45,bc,55.808425457052,37.388807961811,188,23.51
3,41567f28db47bee7,All 25-45 BC,all,25,45,bc,55.674378,37.422364,333,23.51
4,41567f28db47bee7,All 25-45 BC,all,25,45,bc,55.608396,37.766383,54,23.51
...,...,...,...,...,...,...,...,...,...,...
136834,464e9e7d315ba25d,M 25-55,male,25,55,abc,55.907413,37.53523,259,52.66
136835,464e9e7d315ba25d,M 25-55,male,25,55,abc,55.871245,37.410527,231,52.66
136836,464e9e7d315ba25d,M 25-55,male,25,55,abc,55.585678,37.715744,56,52.66
136837,464e9e7d315ba25d,M 25-55,male,25,55,abc,55.74793358781934,37.53652274608613,224,52.66


In [6]:
data = pd.read_excel('/content/drive/My Drive/moscow_admin_division_wth_ao.xlsx')
data

,Субъект,min_longitude,min_latitude,max_longitude,max_latitude
0,Арбат,37.572060,55.743712,37.612228,55.758061
1,Басманный,37.627940,55.750749,37.713525,55.781956
2,Замоскворечье,37.615133,55.720280,37.654211,55.749153
3,Красносельский,37.628386,55.759797,37.688804,55.792981
4,Мещанский,37.612223,55.759120,37.644896,55.797140
...,...,...,...,...,...
120,Крюково,37.144003,55.947901,37.220825,55.999244
121,Матушкино,37.177540,55.986998,37.231254,56.018630
122,Савёлки,37.209283,55.966915,37.268406,56.004668
123,Силино,37.131448,55.980439,37.205714,56.021208


In [7]:
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)
data['min_latitude'] = data['min_latitude'].astype(float)
data['max_latitude'] = data['max_latitude'].astype(float)
data['min_longitude'] = data['min_longitude'].astype(float)
data['max_longitude'] = data['max_longitude'].astype(float)

In [8]:
def get_district(lat, lon, data):
    for index, row in data.iterrows():
        if row['min_latitude'] <= lat <= row['max_latitude'] and row['min_longitude'] <= lon <= row['max_longitude']:
            return row['Субъект']
    return None

# Apply the function to each row in df with tqdm progress bar
tqdm.pandas()
df['district'] = df.progress_apply(lambda row: get_district(row['lat'], row['lon'], data), axis=1)

100%|██████████| 136839/136839 [09:11<00:00, 248.27it/s]


In [9]:
df['district'] = df['district'].fillna('Подмосковье')
df.isna().sum()

lat              0
lon              0
azimuth          0
hash             0
value            0
name             0
gender           0
ageFrom          0
ageTo            0
income           0
id          136839
district         0
dtype: int64

In [10]:
def determine_billboard_side(azimuth):
    """
    Определить сторону билборда ("А" или "Б") на основе азимута.

    azimuth: float - азимут в градусах (0-360)
    return: str - "А" или "Б"
    """
    if 0 <= azimuth <= 90 or 270 <= azimuth <= 360:
        return "А"
    elif 90 < azimuth < 270:
        return "Б"
    else:
        return "Неизвестная сторона"

df['side'] = df['azimuth'].apply(determine_billboard_side)
df

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id,district,side
0,55.573691,37.631423,273,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Западное,А
1,55.584765,37.712454,232,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Восточное,Б
2,55.808425,37.388808,188,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Кунцево,Б
3,55.674378,37.422364,333,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Очаково-Матвеевское,А
4,55.608396,37.766383,54,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Орехово-Борисово Южное,А
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136834,55.907413,37.535230,259,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Северный,Б
136835,55.871245,37.410527,231,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Куркино,Б
136836,55.585678,37.715744,56,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Бирюлёво Восточное,А
136837,55.747934,37.536523,224,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Пресненский,Б


In [11]:
dens = pd.read_excel('/content/drive/My Drive/Плотность районов.xlsx')
dens

,Район,Плотность
0,Академический,18736.02
1,Алексеевский,15196.79
2,Алтуфьевский,17664.00
3,Арбат,16838.39
4,Аэропорт,17313.10
...,...,...
141,Южное Тушино,13725.19
142,Южнопортовый,16418.54
143,Якиманка,5747.71
144,Ярославский,12247.68


In [12]:
dens.rename(columns={'Район': 'district'}, inplace=True)
dens

,district,Плотность
0,Академический,18736.02
1,Алексеевский,15196.79
2,Алтуфьевский,17664.00
3,Арбат,16838.39
4,Аэропорт,17313.10
...,...,...
141,Южное Тушино,13725.19
142,Южнопортовый,16418.54
143,Якиманка,5747.71
144,Ярославский,12247.68


In [13]:
df = pd.merge(df, dens, on='district', how='left')
df[df['Плотность'].isna()].district.unique()

array(['Подмосковье'], dtype=object)

In [14]:
df

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id,district,side,Плотность
0,55.573691,37.631423,273,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Западное,А,10387.90
1,55.584765,37.712454,232,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Восточное,Б,10439.47
2,55.808425,37.388808,188,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Кунцево,Б,9143.90
3,55.674378,37.422364,333,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Очаково-Матвеевское,А,7405.36
4,55.608396,37.766383,54,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Орехово-Борисово Южное,А,21284.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136834,55.907413,37.535230,259,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Северный,Б,3397.47
136835,55.871245,37.410527,231,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Куркино,Б,4216.71
136836,55.585678,37.715744,56,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Бирюлёво Восточное,А,10439.47
136837,55.747934,37.536523,224,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Пресненский,Б,10894.19


In [15]:
df[df['Плотность'].isna()]

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id,district,side,Плотность
134,55.621695,37.421985,121,b3ebb77f965de304,32.09,W 30+ BC,female,30,100,bc,NaN,Подмосковье,Б,NaN
147,55.600074,37.483232,205,b3ebb77f965de304,32.09,W 30+ BC,female,30,100,bc,NaN,Подмосковье,Б,NaN
148,55.632890,37.444042,243,b3ebb77f965de304,32.09,W 30+ BC,female,30,100,bc,NaN,Подмосковье,Б,NaN
151,55.602225,37.471511,293,b3ebb77f965de304,32.09,W 30+ BC,female,30,100,bc,NaN,Подмосковье,А,NaN
156,55.576143,37.354066,146,b3ebb77f965de304,32.09,W 30+ BC,female,30,100,bc,NaN,Подмосковье,Б,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136793,55.997456,37.536735,186,87b45be3fae91c39,59.15,M 25-55,male,25,55,abc,NaN,Подмосковье,Б,NaN
136807,55.571499,37.666427,87,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Подмосковье,А,NaN
136820,55.623283,37.421858,324,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Подмосковье,А,NaN
136824,55.711603,37.371682,82,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Подмосковье,А,NaN


In [16]:
df['Плотность'] = df['Плотность'].fillna(6000)
df

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id,district,side,Плотность
0,55.573691,37.631423,273,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Западное,А,10387.90
1,55.584765,37.712454,232,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Восточное,Б,10439.47
2,55.808425,37.388808,188,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Кунцево,Б,9143.90
3,55.674378,37.422364,333,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Очаково-Матвеевское,А,7405.36
4,55.608396,37.766383,54,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Орехово-Борисово Южное,А,21284.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136834,55.907413,37.535230,259,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Северный,Б,3397.47
136835,55.871245,37.410527,231,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Куркино,Б,4216.71
136836,55.585678,37.715744,56,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Бирюлёво Восточное,А,10439.47
136837,55.747934,37.536523,224,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Пресненский,Б,10894.19


In [17]:
def extract_age(name):
    match = re.search(r'\b(\d{2,3}-\d{2,3}|\d{2,3}\+)\b', name)
    if match:
        return match.group(1)
    return None
def extract_plus_pattern(name):
    match = re.search(r'(\d{1,3}\+)', name)
    if match:
        return match.group(1)
    return None

In [18]:
df['age_range'] = df['name'].apply(extract_age)
df['age_range'] = df['age_range'].fillna(df['name'].apply(extract_plus_pattern))
df.age_range.unique()

array(['25-45', '30-60', '30+', '18+', '25-50', '25-54', '18-45', '25-55',
       '20-49', '30-55', '24+', '35-55', '18-25', '20-40', '25-60',
       '12-24', '20-55', '35-70', '35-54', '25-44', '18-100', '30-99',
       '45-65', '18-50', '14-49', '18-55', '30-50', '15-25', '18-54',
       '35-45', '25+', '35+', '13-25', '18-35', '35-60', '20-45',
       '35-100'], dtype=object)

In [19]:
nedv = pd.read_excel('/content/drive/My Drive/недвижимость.xlsx')
nedv.rename(columns={'Субъект': 'district'}, inplace=True)
df = pd.merge(df, nedv, on='district', how='left')
df

,lat,lon,azimuth,hash,value,name,gender,ageFrom,ageTo,income,id,district,side,Плотность,age_range,Стоимость недвижимости
0,55.573691,37.631423,273,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Западное,А,10387.90,25-45,204613.0
1,55.584765,37.712454,232,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Бирюлёво Восточное,Б,10439.47,25-45,204613.0
2,55.808425,37.388808,188,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Кунцево,Б,9143.90,25-45,289554.0
3,55.674378,37.422364,333,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Очаково-Матвеевское,А,7405.36,25-45,288025.0
4,55.608396,37.766383,54,41567f28db47bee7,23.51,All 25-45 BC,all,25,45,bc,NaN,Орехово-Борисово Южное,А,21284.29,25-45,231958.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136834,55.907413,37.535230,259,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Северный,Б,3397.47,25-55,236200.0
136835,55.871245,37.410527,231,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Куркино,Б,4216.71,25-55,267880.0
136836,55.585678,37.715744,56,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Бирюлёво Восточное,А,10439.47,25-55,204613.0
136837,55.747934,37.536523,224,464e9e7d315ba25d,52.66,M 25-55,male,25,55,abc,NaN,Пресненский,Б,10894.19,25-55,408401.0


In [20]:
df['Стоимость недвижимости'] = df['Стоимость недвижимости'].fillna(200000)

In [21]:
df.drop('id', axis=1, inplace=True)

Преобразование координат в синусы и косинусы

In [22]:
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Преобразование координат в синусоиду и косинусоиду
df['lat_sin'] = np.sin(np.radians(df['lat']))
df['lat_cos'] = np.cos(np.radians(df['lat']))
df['lon_sin'] = np.sin(np.radians(df['lon']))
df['lon_cos'] = np.cos(np.radians(df['lon']))

In [23]:
data_train = df[['hash', 'gender', 'income', 'district', 'side', 'Плотность', 'age_range', 'Стоимость недвижимости', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos', 'value']]
data_train.loc[:, 'side'] = data_train['side'].map({'А': 1, 'Б': 0})
data_train

,hash,gender,income,district,side,Плотность,age_range,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,value
0,41567f28db47bee7,all,bc,Бирюлёво Западное,1,10387.90,25-45,204613.0,0.824854,0.565346,0.610580,0.791955,23.51
1,41567f28db47bee7,all,bc,Бирюлёво Восточное,0,10439.47,25-45,204613.0,0.824963,0.565186,0.611699,0.791091,23.51
2,41567f28db47bee7,all,bc,Кунцево,0,9143.90,25-45,289554.0,0.827163,0.561962,0.607221,0.794533,23.51
3,41567f28db47bee7,all,bc,Очаково-Матвеевское,1,7405.36,25-45,288025.0,0.825846,0.563895,0.607686,0.794177,23.51
4,41567f28db47bee7,all,bc,Орехово-Борисово Южное,1,21284.29,25-45,231958.0,0.825196,0.564846,0.612443,0.790514,23.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136834,464e9e7d315ba25d,male,abc,Северный,0,3397.47,25-55,236200.0,0.828133,0.560532,0.609249,0.792979,52.66
136835,464e9e7d315ba25d,male,abc,Куркино,0,4216.71,25-55,267880.0,0.827779,0.561055,0.607522,0.794303,52.66
136836,464e9e7d315ba25d,male,abc,Бирюлёво Восточное,1,10439.47,25-55,204613.0,0.824972,0.565173,0.611744,0.791055,52.66
136837,464e9e7d315ba25d,male,abc,Пресненский,0,10894.19,25-55,408401.0,0.826569,0.562835,0.609267,0.792965,52.66


In [54]:
grouped = data_train.groupby('hash').agg({
    'gender': 'first',
    'income': 'first',
    'side': 'sum',
    'Плотность': 'mean',
    'age_range': 'first',
    'Стоимость недвижимости': 'mean',
    'lat_sin': lambda x: list(x),
    'lat_cos': lambda x: list(x),
    'lon_sin': lambda x: list(x),
    'lon_cos': lambda x: list(x),
    'value': 'mean'
}).reset_index()
df = pd.get_dummies(grouped, columns=['gender', 'income'])
df

,hash,side,Плотность,age_range,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,value,gender_all,gender_female,gender_male,income_ab,income_abc,income_ac,income_bc,income_c
0,0038e05f5cae8b3e,67,12445.589291,18-55,359028.297872,"[0.8257332260564324, 0.8265830635855338, 0.826...","[0.5640608472376334, 0.5628147466027816, 0.563...","[0.6085085768038558, 0.609620282942925, 0.6080...","[0.793547296609437, 0.7926935792754902, 0.7939...",65.35,True,False,False,False,False,False,True,False
1,004f71aa8a858a53,63,12623.740976,25-50,298663.154472,"[0.8267242270202799, 0.8265830635855338, 0.825...","[0.56260736971508, 0.5628147466027816, 0.56377...","[0.6113252372279786, 0.609620282942925, 0.6097...","[0.791379462917857, 0.7926935792754902, 0.7926...",46.1,True,False,False,False,False,False,True,False
2,006ba0e269ae1acf,22,9107.455200,25-55,238261.140000,"[0.8261166409925421, 0.8279902186489927, 0.828...","[0.563499153038582, 0.5607425414765616, 0.5606...","[0.6102088956473245, 0.6085697185233527, 0.607...","[0.7922405592197818, 0.7935004081261756, 0.794...",49.18,True,False,False,False,False,False,True,False
3,006ece7a187fd5ed,26,13856.946226,20-49,294102.245283,"[0.8259379442932008, 0.8280264183099871, 0.825...","[0.5637610417337487, 0.5606890854838662, 0.564...","[0.6090161755668947, 0.609361035518894, 0.6109...","[0.7931578013849913, 0.7928928858246498, 0.791...",9.71,False,True,False,False,False,False,True,False
4,0082b5dab272280b,12,9810.696000,35-55,226664.400000,"[0.824850339641796, 0.824853990502994, 0.82794...","[0.5653511450353786, 0.5653458183725132, 0.560...","[0.6106384584904987, 0.6105795913402251, 0.611...","[0.7919095106212246, 0.7919548993716774, 0.791...",5.78,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,ff33363bf6ddbe76,1,10367.150000,35-55,417993.666667,"[0.8264368092012171, 0.8268263070979911, 0.826...","[0.5630294844831051, 0.5624573387295241, 0.562...","[0.6093595690586152, 0.6093983492411671, 0.609...","[0.7928940128394834, 0.7928642077570033, 0.792...",0.07,False,False,True,False,False,False,True,False
1543,ff4690b4a2102b03,5,6374.520000,18+,245846.111111,"[0.8254020577054616, 0.8270646513422588, 0.826...","[0.5645453419660726, 0.5621068070216796, 0.562...","[0.6077949818291962, 0.605673315146219, 0.6054...","[0.7940939869204696, 0.7957134128062621, 0.795...",0.53,True,False,False,False,True,False,False,False
1544,ff4a6c6f61fbb78b,19,8328.864043,20-55,222771.914894,"[0.8271224125100167, 0.8280264183099871, 0.827...","[0.5620218098291292, 0.5606890854838662, 0.562...","[0.6076390362626345, 0.609361035518894, 0.6102...","[0.7942133224832084, 0.7928928858246498, 0.792...",17.58,True,False,False,False,False,False,True,False
1545,ff53e40a7e23c8a2,43,12437.055889,25-45,306083.977778,"[0.8265830635855338, 0.8263961202300986, 0.825...","[0.5628147466027816, 0.5630892047168374, 0.564...","[0.609620282942925, 0.6080117379034212, 0.6103...","[0.7926935792754902, 0.7939280361416023, 0.792...",20.04,True,False,False,False,True,False,False,False


In [55]:
df = df.drop(columns=['hash', 'age_range'])

In [56]:
import ast
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val

df['lat_sin'] = df['lat_sin'].apply(safe_literal_eval)
df['lat_cos'] = df['lat_cos'].apply(safe_literal_eval)
df['lon_sin'] = df['lon_sin'].apply(safe_literal_eval)
df['lon_cos'] = df['lon_cos'].apply(safe_literal_eval)

# Вычисление среднего значения для каждого из списков
df['lat_sin'] = df['lat_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lat_cos'] = df['lat_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lon_sin'] = df['lon_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
df['lon_cos'] = df['lon_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)


In [57]:
bool_columns = ['gender_all', 'gender_female', 'gender_male', 'income_ab', 'income_abc', 'income_ac', 'income_bc', 'income_c']
for col in bool_columns:
    df[col] = df[col].astype(int)

# Загрузка и обработка тестового датасета

In [29]:
# Шаг 1: Загрузка данных из JSON файла
file_path = '/content/drive/My Drive/test_data.json'  # укажите путь к вашему JSON файлу

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Шаг 2: Преобразование данных в DataFrame
test = pd.json_normalize(data, 'points', ['hash', 'targetAudience'])

# Преобразование целевой аудитории в отдельные столбцы
test = test.join(pd.json_normalize(test.pop('targetAudience')))
test

,lat,lon,azimuth,hash,name,gender,ageFrom,ageTo,income,id
0,55.70186540076,37.604609012895,125,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN
1,55.774089980695,37.546012401375,226,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN
2,55.749105630591,37.697072315552,128,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN
3,55.656665886902,37.740853428841,346,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN
4,55.735601933067,37.541452478504,144,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN
...,...,...,...,...,...,...,...,...,...,...
27686,55.77237504764,37.426083411045,90,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN
27687,55.585678,37.715744,56,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN
27688,55.620647,37.507592,215,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN
27689,55.621214,37.790481,214,482c582f88422a17,All 30-55 BC,all,30,55,bc,NaN


In [30]:
data_test = test[['hash','name', 'gender', 'ageFrom', 'ageTo','income', 'lat', 'lon', 'azimuth']]
data_test

,hash,name,gender,ageFrom,ageTo,income,lat,lon,azimuth
0,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,55.70186540076,37.604609012895,125
1,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,55.774089980695,37.546012401375,226
2,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,55.749105630591,37.697072315552,128
3,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,55.656665886902,37.740853428841,346
4,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,55.735601933067,37.541452478504,144
...,...,...,...,...,...,...,...,...,...
27686,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,55.77237504764,37.426083411045,90
27687,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,55.585678,37.715744,56
27688,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,55.620647,37.507592,215
27689,482c582f88422a17,All 30-55 BC,all,30,55,bc,55.621214,37.790481,214


In [31]:
data = pd.read_excel('/content/drive/My Drive/moscow_admin_division_wth_ao.xlsx')
data

,Субъект,min_longitude,min_latitude,max_longitude,max_latitude
0,Арбат,37.572060,55.743712,37.612228,55.758061
1,Басманный,37.627940,55.750749,37.713525,55.781956
2,Замоскворечье,37.615133,55.720280,37.654211,55.749153
3,Красносельский,37.628386,55.759797,37.688804,55.792981
4,Мещанский,37.612223,55.759120,37.644896,55.797140
...,...,...,...,...,...
120,Крюково,37.144003,55.947901,37.220825,55.999244
121,Матушкино,37.177540,55.986998,37.231254,56.018630
122,Савёлки,37.209283,55.966915,37.268406,56.004668
123,Силино,37.131448,55.980439,37.205714,56.021208


In [32]:
test['lat'] = test['lat'].astype(float)
test['lon'] = test['lon'].astype(float)
data['min_latitude'] = data['min_latitude'].astype(float)
data['max_latitude'] = data['max_latitude'].astype(float)
data['min_longitude'] = data['min_longitude'].astype(float)
data['max_longitude'] = data['max_longitude'].astype(float)

def get_district(lat, lon, data):
    for index, row in data.iterrows():
        if row['min_latitude'] <= lat <= row['max_latitude'] and row['min_longitude'] <= lon <= row['max_longitude']:
            return row['Субъект']
    return None

# Apply the function to each row in df with tqdm progress bar
tqdm.pandas()
test['district'] = test.progress_apply(lambda row: get_district(row['lat'], row['lon'], data), axis=1)

100%|██████████| 27691/27691 [01:50<00:00, 249.67it/s]


In [33]:
test['district'] = test['district'].fillna('Подмосковье')
test.isna().sum()

lat             0
lon             0
azimuth         0
hash            0
name            0
gender          0
ageFrom         0
ageTo           0
income          0
id          27691
district        0
dtype: int64

In [34]:
def determine_billboard_side(azimuth):
    """
    Определить сторону билборда ("А" или "Б") на основе азимута.

    azimuth: float - азимут в градусах (0-360)
    return: str - "А" или "Б"
    """
    if 0 <= azimuth <= 90 or 270 <= azimuth <= 360:
        return "А"
    elif 90 < azimuth < 270:
        return "Б"
    else:
        return "Неизвестная сторона"

test['side'] = test['azimuth'].apply(determine_billboard_side)
test

,lat,lon,azimuth,hash,name,gender,ageFrom,ageTo,income,id,district,side
0,55.701865,37.604609,125,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Донской,Б
1,55.774090,37.546012,226,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Пресненский,Б
2,55.749106,37.697072,128,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Таганский,Б
3,55.656666,37.740853,346,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Люблино,А
4,55.735602,37.541452,144,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Хамовники,Б
...,...,...,...,...,...,...,...,...,...,...,...,...
27686,55.772375,37.426083,90,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Крылатское,А
27687,55.585678,37.715744,56,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Бирюлёво Восточное,А
27688,55.620647,37.507592,215,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Тёплый Стан,Б
27689,55.621214,37.790481,214,482c582f88422a17,All 30-55 BC,all,30,55,bc,NaN,Братеево,Б


In [35]:
dens = pd.read_excel('/content/drive/My Drive/Плотность районов.xlsx')
dens

,Район,Плотность
0,Академический,18736.02
1,Алексеевский,15196.79
2,Алтуфьевский,17664.00
3,Арбат,16838.39
4,Аэропорт,17313.10
...,...,...
141,Южное Тушино,13725.19
142,Южнопортовый,16418.54
143,Якиманка,5747.71
144,Ярославский,12247.68


In [36]:
dens.rename(columns={'Район': 'district'}, inplace=True)
dens

,district,Плотность
0,Академический,18736.02
1,Алексеевский,15196.79
2,Алтуфьевский,17664.00
3,Арбат,16838.39
4,Аэропорт,17313.10
...,...,...
141,Южное Тушино,13725.19
142,Южнопортовый,16418.54
143,Якиманка,5747.71
144,Ярославский,12247.68


In [37]:
test = pd.merge(test, dens, on='district', how='left')
test[test['Плотность'].isna()].district.unique()

array(['Подмосковье'], dtype=object)

In [38]:
test['Плотность'] = test['Плотность'].fillna(6000)
test

,lat,lon,azimuth,hash,name,gender,ageFrom,ageTo,income,id,district,side,Плотность
0,55.701865,37.604609,125,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Донской,Б,9045.20
1,55.774090,37.546012,226,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Пресненский,Б,10894.19
2,55.749106,37.697072,128,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Таганский,Б,15099.00
3,55.656666,37.740853,346,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Люблино,А,9876.28
4,55.735602,37.541452,144,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Хамовники,Б,10790.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27686,55.772375,37.426083,90,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Крылатское,А,6859.88
27687,55.585678,37.715744,56,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Бирюлёво Восточное,А,10439.47
27688,55.620647,37.507592,215,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Тёплый Стан,Б,17906.40
27689,55.621214,37.790481,214,482c582f88422a17,All 30-55 BC,all,30,55,bc,NaN,Братеево,Б,14392.53


In [39]:
def extract_age(name):
    match = re.search(r'\b(\d{2,3}-\d{2,3}|\d{2,3}\+)\b', name)
    if match:
        return match.group(1)
    return None
def extract_plus_pattern(name):
    match = re.search(r'(\d{1,3}\+)', name)
    if match:
        return match.group(1)
    return None

In [40]:
test['age_range'] = test['name'].apply(extract_age)
test['age_range'] = test['age_range'].fillna(test['name'].apply(extract_plus_pattern))
test.age_range.unique()

array(['25-45', '30+', '25-50', '18+', '18-45', '35-55', '25-55', '30-55',
       '20-49', '30-60', '30-50', '18-54', '20-55', '25-44', '20-40',
       '14-49', '18-100', '30-99', '45-65', '20-45', '18-55', '35-45',
       '25+', '25-54', '25-60', '35+', '25-65', '13-25', '15-25', '40-50',
       '35-60', '24+'], dtype=object)

In [41]:
nedv = pd.read_excel('/content/drive/My Drive/недвижимость.xlsx')
nedv.rename(columns={'Субъект': 'district'}, inplace=True)
test = pd.merge(test, nedv, on='district', how='left')
test

,lat,lon,azimuth,hash,name,gender,ageFrom,ageTo,income,id,district,side,Плотность,age_range,Стоимость недвижимости
0,55.701865,37.604609,125,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Донской,Б,9045.20,25-45,372415.0
1,55.774090,37.546012,226,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Пресненский,Б,10894.19,25-45,408401.0
2,55.749106,37.697072,128,3a94fc3e5c9d8d72,All 25-45 BC,all,25,45,bc,NaN,Таганский,Б,15099.00,25-45,394266.0
3,55.656666,37.740853,346,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Люблино,А,9876.28,30+,226049.0
4,55.735602,37.541452,144,bcc4e68cbec07f33,W 30+ BC,female,30,100,bc,NaN,Хамовники,Б,10790.28,30+,463581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27686,55.772375,37.426083,90,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Крылатское,А,6859.88,25-55,296064.0
27687,55.585678,37.715744,56,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Бирюлёво Восточное,А,10439.47,25-55,204613.0
27688,55.620647,37.507592,215,c9c9b7cbe3a925d4,ALL 25-55 BC,all,25,55,bc,NaN,Тёплый Стан,Б,17906.40,25-55,235441.0
27689,55.621214,37.790481,214,482c582f88422a17,All 30-55 BC,all,30,55,bc,NaN,Братеево,Б,14392.53,30-55,229188.0


In [42]:
test['Стоимость недвижимости'] = test['Стоимость недвижимости'].fillna(200000)

In [43]:
test.drop('id', axis=1, inplace=True)

In [44]:
test['lat'] = test['lat'].astype(float)
test['lon'] = test['lon'].astype(float)

# Преобразование координат в синусоиду и косинусоиду
test['lat_sin'] = np.sin(np.radians(test['lat']))
test['lat_cos'] = np.cos(np.radians(test['lat']))
test['lon_sin'] = np.sin(np.radians(test['lon']))
test['lon_cos'] = np.cos(np.radians(test['lon']))

In [46]:
data_test = test[['hash', 'gender', 'income', 'district', 'side', 'Плотность', 'age_range', 'Стоимость недвижимости', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos']]
data_test.loc[:, 'side'] = data_test['side'].map({'А': 1, 'Б': 0})
data_test

,hash,gender,income,district,side,Плотность,age_range,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos
0,3a94fc3e5c9d8d72,all,bc,Донской,0,9045.20,25-45,372415.0,0.826117,0.563499,0.610209,0.792241
1,3a94fc3e5c9d8d72,all,bc,Пресненский,0,10894.19,25-45,408401.0,0.826826,0.562457,0.609398,0.792864
2,3a94fc3e5c9d8d72,all,bc,Таганский,0,15099.00,25-45,394266.0,0.826581,0.562818,0.611487,0.791255
3,bcc4e68cbec07f33,female,bc,Люблино,1,9876.28,30+,226049.0,0.825672,0.564151,0.612091,0.790787
4,bcc4e68cbec07f33,female,bc,Хамовники,0,10790.28,30+,463581.0,0.826448,0.563013,0.609335,0.792913
...,...,...,...,...,...,...,...,...,...,...,...,...
27686,c9c9b7cbe3a925d4,all,bc,Крылатское,1,6859.88,25-55,296064.0,0.826809,0.562482,0.607737,0.794138
27687,c9c9b7cbe3a925d4,all,bc,Бирюлёво Восточное,1,10439.47,25-55,204613.0,0.824972,0.565173,0.611744,0.791055
27688,c9c9b7cbe3a925d4,all,bc,Тёплый Стан,0,17906.40,25-55,235441.0,0.825317,0.564670,0.608867,0.793273
27689,482c582f88422a17,all,bc,Братеево,0,14392.53,30-55,229188.0,0.825323,0.564661,0.612776,0.790257


In [47]:
grouped = data_test.groupby('hash').agg({
    'gender': 'first',
    'income': 'first',
    'side': 'sum',
    'Плотность': 'mean',
    'age_range': 'first',
    'Стоимость недвижимости': 'mean',
    'lat_sin': lambda x: list(x),
    'lat_cos': lambda x: list(x),
    'lon_sin': lambda x: list(x),
    'lon_cos': lambda x: list(x)
}).reset_index()
test = pd.get_dummies(grouped, columns=['gender', 'income'])
test

,hash,side,Плотность,age_range,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,gender_female,gender_male,income_ab,income_abc,income_bc,income_c
0,00e38c48d1fd1854,9,11696.545556,25-55,409811.333333,"[0.8261166409925421, 0.8264916001800454, 0.826...","[0.563499153038582, 0.56294905171945, 0.562765...","[0.6102088956473245, 0.6108895224849743, 0.610...","[0.7922405592197818, 0.7917158526378514, 0.792...",True,False,False,False,False,True,False
1,00ebb4dffe97be61,23,12846.807414,30-55,401050.172414,"[0.8263957460029147, 0.8266774956704327, 0.826...","[0.5630897539365161, 0.562676033035051, 0.5627...","[0.6105758327969693, 0.610947145392, 0.6109352...","[0.7919577971106083, 0.7916713873428586, 0.791...",False,True,False,False,False,True,False
2,01768dfc4f866f35,164,12579.312970,25+,322962.427273,"[0.8265577058171314, 0.8265183109014451, 0.826...","[0.5628519867197063, 0.5629098344714027, 0.563...","[0.6101451639012676, 0.6093515812020489, 0.610...","[0.7922896433551907, 0.792900151649981, 0.7922...",True,False,False,False,False,False,True
3,022d43449e12ab00,14,10268.659310,25-50,230001.689655,"[0.8279902186489927, 0.8279902186489927, 0.828...","[0.5607425414765616, 0.5607425414765616, 0.560...","[0.6085697185233527, 0.6085697185233527, 0.609...","[0.7935004081261756, 0.7935004081261756, 0.792...",True,False,False,False,True,False,False
4,02c7777339f8c82b,14,13081.340000,25-55,413156.960000,"[0.8263964564801171, 0.8267725687687201, 0.826...","[0.5630887112321699, 0.562536327299466, 0.5630...","[0.6105351519400387, 0.610225495855177, 0.6107...","[0.7919891591717363, 0.7922277729342132, 0.791...",True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,fe98838a2a6aca37,87,12434.803176,18-55,304649.188235,"[0.8267242270202799, 0.8265830635855338, 0.825...","[0.56260736971508, 0.5628147466027816, 0.56377...","[0.6113252372279786, 0.609620282942925, 0.6097...","[0.791379462917857, 0.7926935792754902, 0.7926...",True,False,False,False,False,True,False
384,feb0969326308d04,47,10848.058469,25-50,225891.418367,"[0.824853990502994, 0.8280413512548717, 0.8279...","[0.5653458183725132, 0.5606670318575958, 0.560...","[0.6105795913402251, 0.6077095064351778, 0.611...","[0.7919548993716774, 0.7941594020020871, 0.791...",True,False,False,False,True,False,False
385,fef77e78d322e465,116,12351.421327,45-65,322974.207965,"[0.8267242270202799, 0.8265830635855338, 0.825...","[0.56260736971508, 0.5628147466027816, 0.56377...","[0.6113252372279786, 0.609620282942925, 0.6097...","[0.791379462917857, 0.7926935792754902, 0.7926...",True,False,False,False,False,True,False
386,ff93334ac1156a3d,11,10631.035429,25-55,358973.114286,"[0.8278974670796864, 0.8262207073503547, 0.826...","[0.5608794736866732, 0.5633465565222525, 0.562...","[0.6079905663232826, 0.6071357394579115, 0.609...","[0.793944249467111, 0.7945981335699794, 0.7924...",True,False,False,False,True,False,False


In [48]:
test = test.drop(columns=['hash', 'age_range'])

In [49]:
import ast
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val

test['lat_sin'] = test['lat_sin'].apply(safe_literal_eval)
test['lat_cos'] = test['lat_cos'].apply(safe_literal_eval)
test['lon_sin'] = test['lon_sin'].apply(safe_literal_eval)
test['lon_cos'] = test['lon_cos'].apply(safe_literal_eval)

# Вычисление среднего значения для каждого из списков
test['lat_sin'] = test['lat_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
test['lat_cos'] = test['lat_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
test['lon_sin'] = test['lon_sin'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)
test['lon_cos'] = test['lon_cos'].apply(lambda x: np.mean(x) if isinstance(x, list) else x)


In [69]:
test['side'] = pd.to_numeric(test['side'])

In [76]:
df['side'] = pd.to_numeric(df['side'])

In [52]:
test['income_ac'] = 0
bool_columns = ['gender_all', 'gender_female', 'gender_male', 'income_ab', 'income_abc', 'income_ac', 'income_bc', 'income_c']
for col in bool_columns:
    test[col] = test[col].astype(int)

# Модели

In [81]:
display(df)
test = test[['side', 'Плотность', 'Стоимость недвижимости', 'lat_sin', 'lat_cos', 'lon_sin', 'lon_cos', 'gender_all', 'gender_female', 'gender_male', 'income_ab', 'income_abc', 'income_ac', 'income_bc', 'income_c']]

,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,value,gender_all,gender_female,gender_male,income_ab,income_abc,income_ac,income_bc,income_c
0,67,12445.589291,359028.297872,0.826584,0.562812,0.610297,0.792171,65.35,1,0,0,0,0,0,1,0
1,63,12623.740976,298663.154472,0.826670,0.562685,0.610007,0.792394,46.1,1,0,0,0,0,0,1,0
2,22,9107.455200,238261.140000,0.826760,0.562548,0.609964,0.792423,49.18,1,0,0,0,0,0,1,0
3,26,13856.946226,294102.245283,0.826492,0.562946,0.610069,0.792346,9.71,0,1,0,0,0,0,1,0
4,12,9810.696000,226664.400000,0.826459,0.562992,0.610391,0.792095,5.78,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,1,10367.150000,417993.666667,0.826575,0.562826,0.609355,0.792897,0.07,0,0,1,0,0,0,1,0
1543,5,6374.520000,245846.111111,0.827062,0.562107,0.606849,0.794816,0.53,1,0,0,0,1,0,0,0
1544,19,8328.864043,222771.914894,0.826130,0.563468,0.611244,0.791423,17.58,1,0,0,0,0,0,1,0
1545,43,12437.055889,306083.977778,0.826523,0.562901,0.610295,0.792173,20.04,1,0,0,0,1,0,0,0


In [82]:
from os import X_OK
# Разделение данных на признаки (X) и целевую переменную (y)
X_train = df.drop(columns=['value'])
y_train = df['value']
X_test = test

In [83]:
X_test

,side,Плотность,Стоимость недвижимости,lat_sin,lat_cos,lon_sin,lon_cos,gender_all,gender_female,gender_male,income_ab,income_abc,income_ac,income_bc,income_c
0,9,11696.545556,409811.333333,0.826578,0.562823,0.610199,0.792247,1,0,0,0,0,0,1,0
1,23,12846.807414,401050.172414,0.826637,0.562735,0.610417,0.792080,0,1,0,0,0,0,1,0
2,164,12579.312970,322962.427273,0.826596,0.562795,0.610161,0.792276,1,0,0,0,0,0,0,1
3,14,10268.659310,230001.689655,0.826806,0.562483,0.610348,0.792129,1,0,0,0,1,0,0,0
4,14,13081.340000,413156.960000,0.826628,0.562748,0.610379,0.792109,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,87,12434.803176,304649.188235,0.826676,0.562677,0.610008,0.792393,1,0,0,0,0,0,1,0
384,47,10848.058469,225891.418367,0.826463,0.562987,0.610623,0.791916,1,0,0,0,1,0,0,0
385,116,12351.421327,322974.207965,0.826652,0.562712,0.610081,0.792337,1,0,0,0,0,0,1,0
386,11,10631.035429,358973.114286,0.826632,0.562742,0.609474,0.792804,1,0,0,0,1,0,0,0


## 1 модель

In [78]:
# Создание и обучение модели CatBoost
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, eval_metric='RMSE', random_seed=42, verbose=100)
model.fit(X_train, y_train, early_stopping_rounds=50)

# Предсказание значений
y_pred_cb = model.predict(X_test)
len(y_pred_cb)

0:	learn: 22.5774225	total: 2.86ms	remaining: 2.86s
100:	learn: 9.6650752	total: 212ms	remaining: 1.88s
200:	learn: 7.3187520	total: 434ms	remaining: 1.73s
300:	learn: 6.2064106	total: 637ms	remaining: 1.48s
400:	learn: 5.5231232	total: 832ms	remaining: 1.24s
500:	learn: 5.1159494	total: 1.04s	remaining: 1.03s
600:	learn: 4.7707868	total: 1.24s	remaining: 826ms
700:	learn: 4.5344866	total: 1.46s	remaining: 622ms
800:	learn: 4.3597882	total: 1.66s	remaining: 412ms
900:	learn: 4.2248652	total: 1.86s	remaining: 204ms
999:	learn: 4.1186302	total: 2.06s	remaining: 0us


388

In [92]:
y_pred_cb = np.abs(y_pred_cb)
file_path = '/content/drive/My Drive/sample_submission.csv'
sam1 = pd.read_csv(file_path)
sam1['value'] = y_pred_cb
sam1

,hash,value
0,3a94fc3e5c9d8d72,30.002550
1,bcc4e68cbec07f33,0.218565
2,d8da298a5c2b2f27,72.924474
3,5c75b1a1602e33d7,32.858765
4,609bcaed570f5815,0.925866
...,...,...
383,4e154970e1c6ae2b,32.819188
384,3c15071c77f49893,81.872363
385,c4576f9dcda42732,58.223146
386,c9c9b7cbe3a925d4,46.756891


In [93]:
sam1.to_csv('submission2.csv', index=False)

## 2 модель

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   side                    1547 non-null   int64  
 1   Плотность               1547 non-null   float64
 2   Стоимость недвижимости  1547 non-null   float64
 3   lat_sin                 1547 non-null   float64
 4   lat_cos                 1547 non-null   float64
 5   lon_sin                 1547 non-null   float64
 6   lon_cos                 1547 non-null   float64
 7   value                   1547 non-null   object 
 8   gender_all              1547 non-null   int64  
 9   gender_female           1547 non-null   int64  
 10  gender_male             1547 non-null   int64  
 11  income_ab               1547 non-null   int64  
 12  income_abc              1547 non-null   int64  
 13  income_ac               1547 non-null   int64  
 14  income_bc               1547 non-null   

In [129]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)


In [130]:
from sklearn.linear_model import LinearRegression

catboost_model = CatBoostRegressor(iterations=1000, learning_rate=0.05, depth=10, verbose=100)
lgb_model = LGBMRegressor(n_estimators=1000, learning_rate=0.05, max_depth=-1)
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=10, objective='reg:squarederror')

# Определение мета-модели
meta_model = Lasso(alpha=0.1)

# Определение стэкинг-модели
stacking_model = StackingRegressor(
    estimators=[
        ('catboost', catboost_model),
        ('lightgbm', lgb_model),
        ('xgboost', xgb_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

# Обучение стэкинг-модели
stacking_model.fit(X_train, y_train)
y_pred_stack = stacking_model.predict(X_test)

In [131]:
y_pred_stack

array([26.30755479, 26.29414356, 27.21979057, 26.17310482, 27.21979057,
       26.30755479, 26.30755479, 27.30848764, 27.21979057, 26.30755479,
       27.07649461, 26.30755479, 24.38047893, 26.17310482, 28.40628545,
       27.21979057, 23.8370198 , 27.30848764, 26.30755479, 19.20263863,
       27.27704342, 26.30755479, 26.30755479, 18.32459793, 26.30755479,
       26.17310482, 26.34333423, 26.30755479, 26.30755479, 26.30755479,
       26.30755479, 26.30755479, 26.30755479, 26.17310482, 25.95256839,
       26.30755479, 27.21979057, 26.30755479, 26.30755479, 26.30755479,
       26.30755479, 26.17310482, 26.30755479, 26.30755479, 26.17310482,
       26.29414356, 28.37154942, 26.30755479, 26.17310482, 26.00377539,
       27.30848764, 19.20263863, 26.17310482, 26.30755479, 27.21979057,
       27.31934242, 26.17310482, 26.30755479, 26.17310482, 24.38047893,
       26.30755479, 24.38047893, 23.8370198 , 19.20263863, 26.30755479,
       26.30755479, 26.17310482, 26.30755479, 27.27209576, 26.00

In [132]:
file_path = '/content/drive/My Drive/sample_submission.csv'
sam = pd.read_csv(file_path)
sam

,hash,value
0,3a94fc3e5c9d8d72,10
1,bcc4e68cbec07f33,10
2,d8da298a5c2b2f27,10
3,5c75b1a1602e33d7,10
4,609bcaed570f5815,10
...,...,...
383,4e154970e1c6ae2b,10
384,3c15071c77f49893,10
385,c4576f9dcda42732,10
386,c9c9b7cbe3a925d4,10


In [133]:
y_pred_stack

array([26.30755479, 26.29414356, 27.21979057, 26.17310482, 27.21979057,
       26.30755479, 26.30755479, 27.30848764, 27.21979057, 26.30755479,
       27.07649461, 26.30755479, 24.38047893, 26.17310482, 28.40628545,
       27.21979057, 23.8370198 , 27.30848764, 26.30755479, 19.20263863,
       27.27704342, 26.30755479, 26.30755479, 18.32459793, 26.30755479,
       26.17310482, 26.34333423, 26.30755479, 26.30755479, 26.30755479,
       26.30755479, 26.30755479, 26.30755479, 26.17310482, 25.95256839,
       26.30755479, 27.21979057, 26.30755479, 26.30755479, 26.30755479,
       26.30755479, 26.17310482, 26.30755479, 26.30755479, 26.17310482,
       26.29414356, 28.37154942, 26.30755479, 26.17310482, 26.00377539,
       27.30848764, 19.20263863, 26.17310482, 26.30755479, 27.21979057,
       27.31934242, 26.17310482, 26.30755479, 26.17310482, 24.38047893,
       26.30755479, 24.38047893, 23.8370198 , 19.20263863, 26.30755479,
       26.30755479, 26.17310482, 26.30755479, 27.27209576, 26.00

In [135]:
sam['value'] = y_pred_stack

In [136]:
sam.to_csv('submission9.csv', index=False)

## 3 модель

In [115]:
# Обучение XGBoost модели
xgb_params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.01,
    'n_estimators': 1000,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'rmse'
}
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train)

# Обучение CatBoost модели
catboost_params = {
    'iterations': 1500,
    'learning_rate': 0.05,
    'depth': 11,
    'eval_metric': 'RMSE',
    'verbose': 200
}
catboost_model = CatBoostRegressor(**catboost_params)
catboost_model.fit(X_train, y_train)


xgb_test_pred = xgb_model.predict(X_test)
catboost_test_pred = catboost_model.predict(X_test)


# Взвешенное усреднение предсказаний
weights = [0.6, 0.4]  # Вес для каждой модели, можно настроить
ensemble_test_pred = (weights[0] * xgb_test_pred +
                      weights[1] * catboost_test_pred)


0:	learn: 23.1517378	total: 40.1ms	remaining: 1m
200:	learn: 5.9930837	total: 9.56s	remaining: 1m 1s
400:	learn: 4.5191901	total: 18.8s	remaining: 51.5s
600:	learn: 4.0227012	total: 26.9s	remaining: 40.3s
800:	learn: 3.8255506	total: 36.7s	remaining: 32s
1000:	learn: 3.7368979	total: 44.2s	remaining: 22.1s
1200:	learn: 3.6927793	total: 54.2s	remaining: 13.5s
1400:	learn: 3.6680743	total: 1m 4s	remaining: 4.55s
1499:	learn: 3.6594836	total: 1m 8s	remaining: 0us


In [121]:
len(ensemble_test_pred)

388

# Нами было принято решениее в качестве основной использовать модель 2
